# Attention 뽀개기

https://pozalabs.github.io/transformer/ 를 읽어보며 정리한 jupyter notebook 입니다.

이장에선 transformer 논문을 요약하면서 추가적인 기법을 설명하도록 합니다.

## Why?

### Long-term dependency problem
- sequence data를 처리하기 위해 이전까지 많이 쓰이던 model 은 recurrent model 이였습니다. recurrent model 은 t번째에 대한 output 을 만들기 위해 t번째 input과 t-1 번쨰 hidden state 값을 이용했습니다. 이렇게 한다면 자연스럽게 문자으이 순차적인 특성이 유지가 되었습니다. 문장을 쓸 떄 뒤의 단어부터 쓰지 않고 처음부터 차례차례 쓰는 것과 마찬가지 인것입니다.
- 하지만 recurrent model 의 경우 많은 개선점이 있었음에도 long-term-dependency에 취약하다는 단점이 있었습니다. 예를 들어 "저는 언어학을 좋아하고, 인공지능중에서도 딥러닝을 배우고 있고 자연어 처리에 관심이 많습니다" 라는 문장을 만드는 것이 model의 task라고 합시다 이때 '자연어'단어를 만드는데 '언어학'이라는 단어는 중요한 단서입니다.
- 그러나 두 단어 사이의 거리가 가깝지 않으므로 model은 앞의 '언어학'이 라는 단어를이용해 '자연어'라는 단어를 만들지 못하고 , 언어학 보다 가까운 단어인 '딥러닝'을 보고 '이미지'를 만들 수도 있는것 입니다. 이처럼, 어떤정보와 다른 정보 사이의 거리가 멀 때 해당 정보를 이용하지 못하는것이 long-term dependecy problem 입니다.
- recurrent model은 순차적인 특성이 유지되는 뛰어난 장점이 있었음에도, long-term dependency problem 이라는 단점을 가지고 있습니다.
- 이와 달리 transformer는 recurrence 를 사용하지 않고 대신 attention mechanism만을 사용해 input 과 output의 dependency를 포착해냈습니다.

### Parallelization 
- recurrent model 은 학습 시 , t번째 hidden state를 얻기 위해서 t-1번째 hidden state가 필요 했습니다. 즉, 순서대로 계산될 필요가 있었습니다. 그래서 병렬 처리를 할 수 없었고 계산 속도가 느렸습니다.
( hidden state를 이용해 순서대로 계산됬기 때문에 속도가 느렸다 )
- 하지만 transformer에서는 학습 시 encoder에서는 각각의 position에 대해, 즉 각각의 단어에 대해 attention을 해주기만 하고, decoder에서는 masking 기법을 이용해 병렬 처리가 가능해집니다. (masking에 대해선 아래서 알려드림)


## Model Architecture

### Encoder and Decoder structure
![](https://pozalabs.github.io/assets/images/encoder-decoder.png)

- encoder는 input sequence (x1,...xn)에 대해 서로 다른 representation 인 z = (z1,....zn)으로 바꿔줍니다.
- decoder 는 z를 받아, output sequence(y1,...yn)를 하나씩 만들어 냅니다.
- 각각의 step에서 다음 symbol을 만들 때 이전에 만들어진 output(symbol)을 이용합니다. 예를 들어 "저는 사람입니다"라는 문장에서 사람입니다를 만들 때 '저는'이라는 symbol을 이용하는 겁니다.이런 특성을 auto-regressive하다고 합니다.


### Encoder and Decoder stacks 
![Encoder and Decoder stacks](https://pozalabs.github.io/assets/images/archi2.png)

### Encoder 
- N개의 동일한 Layer 로 구성돼 있습니다. input(x) 가 첫번째 layer에 들어가게 되고, layer(x)는 다시 layer에 들어가는 식입니다.
- 그리고 각각의 layer는 두개의 sub-layer, multi-head self-attention mechanism과 position-wise fully connected feed-foward network를 가지고 있습니다.
- 이때 두 개의 sub-layer에 residual connection을 이요합니다.  residual connection은 input을 output으로 그대로 전달하는 것을 말합니다. 이때 sub-layer의 output dimension을 embedding dimention과 맞춰줍니다. x+Sublayer(x)를 하기 위해서, 즉 residual connection 을 하기 위해서는 두 값의 차원을 맞춰야 할 필요가 있습니다. 그 후에  layer normalization을 적용합니다.

### Decoder 
- 역시 N개의 동일한 layer로 이루어져 있습니다.
- encoder와 달리 encoder의 결과에 multi-head attention 을 수행할 sub-layer를 추가합니다. 
- 마찬가지로 sub-layer에 residual connection을 사용한뒤, layer normalization 을 해줍니다.
- decoder에서는 encoder 와 달리 순차적으로 결과를 만들어내야 하기 때문에, self-attention을 변형합니다. 바로 masking을 해주는것입니다.  masking을 통해 position i 보다 이후에 있는 posititon에 attention을 주지 못하게 합니다. 즉, position i에 대한 예측은 미리 알고 있는 output들에만 의존을 하는 것입니다.  
![decoder_masking](https://pozalabs.github.io/assets/images/masking.png)

- 위의 예시를 보면, a를 예측할 때는 a이후에 있는 b,c에는 attention이 주어지지 않는 것 입니다. 그리고 b를 예측할 떄는 b이전에 있는 a만 attention이 주어질 수 있고 이후에 있는 c는 attention이 주어지지 않습니다.

### Embedding and Softmax
- embedding 값을 고정시키지 않고, 학습을 하면서 embedding값이 변경되는 learned embedding을 사용했습니다. 이때 input과 output은 같은 embedding layer를 사용합니다.
- 또한 decoder output을 다음 token의 확률로 바꾸기 위해 learned linear transformer과 softmax function을 사용했습니다. learned linear transformation을 사용했다는 것은 decoder output에 weight matrix W를 곱해주는데, 이때 W가 학습된다는 것입니다. 

### Attention
- attention은 단어의 의미처럼 특정 정보에 좀 더 주의를 기울이는 것입니다.
- 예를 들어 model이 수행해야 하는 task가 번역이라고 해봅시다. source는 영어이고, target은 한국어 입니다. "Hi my name is HSW" 라는 문장과 대응되는 "안녕, 내이름은 승우야"라는 문장이 있습니다. model이 (이름은)이라는 token을 decode할때 source에서 가장 중요한 것은 name 입니다.
- 그렇다면, source의 모든 token이 비슷한 중요도를 갖기 보다는 name이 더 큰 중요도를 가지면 되겠습니다. 이때, 더 큰 중요도를 갖게 만드는 방법이 바로 attention입니다. 

### Scaled Dot-Product Attention
![Scaled Dot-Product Attention](https://pozalabs.github.io/assets/images/sdpa.PNG)
- 논문에선 attention을 Scaled Dot-Product-Attention이라고 부릅니다. 수식을 살펴보면 이렇게 부르는 이유가 있습니다. 
![](../datasets/img/20191020_234348.png)

- 먼저 input은 dk dimention의 query와 key들, dv dimention의 value들로 이루어져 있습니다.
- 이때 모든 query와 key에 대한 dot-product를 계산하고 각각을 sqrt(dk)로 나누어 줍니다. dot-product를 하고 sqrt(dk)로 scaling 을 해주기 때문에 Scaled Dot-Product Attention 이라고 합니다. 그리고 여기에 softmax를 적용해 value들에 weights를 얻어냅니다.
- Key와 Value는 Attention이 이루어지는 위치에 상관없이 같은 값을 갖게 됩니다. 